In [1]:
import os

In [2]:
%pwd

'e:\\work\\DS\\Wine-Quality-Prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\work\\DS\\Wine-Quality-Prediction'

In [5]:
import pandas as pd

In [6]:
data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [8]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [9]:
data.shape

(1599, 12)

In [10]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir:Path
    STATUS_FILE: Path
    unzip_data_dir:Path
    all_schema: dict

In [12]:
from src.Wine_Quality_Prediction.constants import *
from src.Wine_Quality_Prediction.utils.common import read_yaml, create_directories

In [23]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):  
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_dir,
            all_schema = schema,
        )
        return data_validation_config

In [14]:
import os
from Wine_Quality_Prediction import logger

In [40]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config 

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None
            data = pd.read_csv(self.config.unzip_data_dir)
            all_columns = list(data.columns)
            all_schema = self.config.all_schema.keys()
            os.makedirs(os.path.dirname(self.config.STATUS_FILE), exist_ok=True)

            for col in all_columns:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"Validation Status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"Validation Status: {validation_status}")
            return validation_status
        except Exception as e:
            raise e
        

In [42]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e


[2025-12-23 09:12:27,007]: INFO: yaml file: config\config.yaml loaded successfully
[2025-12-23 09:12:27,007]: INFO: yaml file: params.yaml loaded successfully
[2025-12-23 09:12:27,007]: INFO: yaml file: schema.yaml loaded successfully
[2025-12-23 09:12:27,007]: INFO: created directory at: artifacts
